In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd 
import json
import os
import shutil
import tempfile

# Indexes data for tracking - DO NOT RUN IF YOU ARE NOT UPDATING train_data.json

'''

data_file  = 'data/train_data.json'
# make a backup first
shutil.copy2(data_file, data_file + '.bak')

# read, add indexes only if needed, and write atomically to avoid corruption
with open(data_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

if not isinstance(data, list):
    raise ValueError("Expected JSON file to contain a list of entries")

for i, entry in enumerate(data):
    if isinstance(entry, dict):
        # only set/update the index if it's missing or out of sync
        if entry.get('index') != i:
            entry['index'] = i

# atomic write to a temp file in the same directory, then replace
dirpath = os.path.dirname(data_file) or '.'
temp_fp = None
try:
    with tempfile.NamedTemporaryFile('w', delete=False, dir=dirpath, encoding='utf-8') as tf:
        json.dump(data, tf, ensure_ascii=False, indent=2)
        temp_fp = tf.name
    os.replace(temp_fp, data_file)
    temp_fp = None
except Exception:
    # clean up temp file on error and re-raise
    if temp_fp and os.path.exists(temp_fp):
        os.remove(temp_fp)
    raise

'''

In [7]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
import json
import os


In [8]:
# Initialize the model with GPU support if available
model = SentenceTransformer("google/embeddinggemma-300m")

data_file = 'data/train_data.json'
# Load the data
with open(data_file, 'r') as f:
    data = json.load(f)

# Extract fields to be encoded
metric_names = [entry['metric_name'] for entry in data]
user_prompts = [entry['user_prompt'] for entry in data]
responses = [entry['response'] for entry in data]
system_prompts = [entry['system_prompt'] if entry['system_prompt'] else '' for entry in data]
scores = [float(entry['score']) for entry in data]
index = [int(entry['index']) for entry in data]

# Batch encode fields
metric_name_embeddings = model.encode(metric_names, batch_size=128)
user_prompt_embeddings = model.encode(user_prompts, batch_size=128)
response_embeddings = model.encode(responses, batch_size=128)
system_prompt_embeddings = model.encode(system_prompts, batch_size=128)

# Combine embeddings and scores into a DataFrame
transformed_data = {
    'index': index,
    'metric_name': metric_name_embeddings.tolist(),
    'user_prompt': user_prompt_embeddings.tolist(),
    'response': response_embeddings.tolist(),
    'system_prompt': system_prompt_embeddings.tolist(),
    'score': scores
}

df = pd.DataFrame(transformed_data)

# Save the DataFrame to a file
output_file = 'data/train_data_conv.json'
df.to_json(output_file, orient='records', lines=True)

# Display the first few rows of the DataFrame
df.head()

KeyboardInterrupt: 